In [1]:
!pip install datasets==3.6.0 -q
!pip install transformers -q
!pip install torchaudio -q
!pip install librosa -q
!pip install jiwer -q
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
WAV_DIR = "/content/drive/MyDrive/project/Bert_Fair/Fair/"

In [52]:
Whisper_DIR = "/content/drive/MyDrive/project/Whisper_Fair/Fair/"

In [106]:
CLINICAL_TERMS = "clinical_terms.json"

In [107]:
import json
import re

def load_medical_terms(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    if not isinstance(data, dict):
        raise ValueError("Expected JSON object mapping term count")

    normalized_terms = set()

    for term in data.keys():
        term = term.lower()
        term = re.sub(r"[^\w\s]", "", term)
        term = re.sub(r"\s+", " ", term).strip()
        if term:
            normalized_terms.add(term)

    return normalized_terms

In [108]:
medical_terms = load_medical_terms(CLINICAL_TERMS)

In [109]:
print(f"Loaded {len(medical_terms)} medical terms")
print(sorted(list(medical_terms))[:200])

Loaded 1005 medical terms
['100mg', '10fold', '10mg', '10th', '1149', '125gm50ml', '12cchr', '135', '1430', '150cck', '1530', '20mg', '25', '250mg', '25mg', '25mg5ml', '28yo', '2bilaterally', '300mg30mg', '30mg100ml', '30ml', '35cc', '40mgml', '45mg', '4cm', '4grams', '50ccs', '5mg', '5mins', '60s', '65the', '6756', '70s', '75mg', '8year', '925', 'aas', 'abcrosssectional', 'abd', 'abdomen', 'ablation', 'absence', 'abutting', 'accelerationdeceleration', 'acetaminophen', 'acetate', 'achike', 'acute', 'adenocarcinoma', 'adenoids', 'adeq', 'adequacy', 'administered', 'administering', 'administration', 'admitting', 'adolescence', 'adrenal', 'affected', 'afib', 'afp', 'akukwe', 'albuterol', 'alcohol', 'alkaline', 'allergies', 'alveolar', 'alzheimers', 'ambient', 'amputation', 'amts', 'anaesthetic', 'anaphylactic', 'androgen', 'anemia', 'aneurysm', 'angio', 'anisopoikilocytosis', 'ankle', 'anomalies', 'anovulation', 'anterior', 'anteroseptal', 'antibiotic', 'antibodies', 'anticipate', 'antidn

In [110]:
medical_terms = {
  term for term in medical_terms
  if len(term) > 3 and not term.isdigit()
}

In [111]:
print(f"Loaded {len(medical_terms)} medical terms")
print(sorted(list(medical_terms))[:200])

Loaded 976 medical terms
['100mg', '10fold', '10mg', '10th', '125gm50ml', '12cchr', '150cck', '20mg', '250mg', '25mg', '25mg5ml', '28yo', '2bilaterally', '300mg30mg', '30mg100ml', '30ml', '35cc', '40mgml', '45mg', '4grams', '50ccs', '5mins', '65the', '75mg', '8year', 'abcrosssectional', 'abdomen', 'ablation', 'absence', 'abutting', 'accelerationdeceleration', 'acetaminophen', 'acetate', 'achike', 'acute', 'adenocarcinoma', 'adenoids', 'adeq', 'adequacy', 'administered', 'administering', 'administration', 'admitting', 'adolescence', 'adrenal', 'affected', 'afib', 'akukwe', 'albuterol', 'alcohol', 'alkaline', 'allergies', 'alveolar', 'alzheimers', 'ambient', 'amputation', 'amts', 'anaesthetic', 'anaphylactic', 'androgen', 'anemia', 'aneurysm', 'angio', 'anisopoikilocytosis', 'ankle', 'anomalies', 'anovulation', 'anterior', 'anteroseptal', 'antibiotic', 'antibodies', 'anticipate', 'antidnaase', 'antigens', 'antihypertensives', 'antiinflammatory', 'antineoplastic', 'antistreptohyaluronidas

In [112]:
from datasets import load_dataset_builder, Split, Audio
import jiwer

builder = load_dataset_builder(
    "tobiolatunji/afrispeech-200",
    name="pidgin"
)

# Run dataset script
builder.download_and_prepare()

# Create dataset
pidgin_speech = builder.as_dataset()

pidgin_speech = pidgin_speech.cast_column("audio", Audio(sampling_rate=16_000))

train_ds = pidgin_speech[Split.TRAIN]
dev_ds   = pidgin_speech[Split.VALIDATION]
test_ds  = pidgin_speech[Split.TEST]

In [113]:
import re

def normalize_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)   # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()
    return text

def normalize_transcript(batch):
    batch["transcript"] = normalize_text(batch["transcript"])
    return batch

test_ds = test_ds.map(normalize_transcript)

In [114]:
test_ds[0]

{'speaker_id': '07314a953e477ce489382c19e7603432',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/ec1b6cdcc439d2ab20deba5dfdf6b3615c35553435f484b4fc8f3b975f7513cf/d4336ca9-6f12-4115-9a5f-e5906b05e05e/e018849df98411babb1145697096baeb.wav',
 'audio_id': 'd4336ca9-6f12-4115-9a5f-e5906b05e05e/e018849df98411babb1145697096baeb',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/ec1b6cdcc439d2ab20deba5dfdf6b3615c35553435f484b4fc8f3b975f7513cf/d4336ca9-6f12-4115-9a5f-e5906b05e05e/e018849df98411babb1145697096baeb.wav',
  'array': array([-0.0029954 , -0.00280054,  0.00355052, ...,  0.00048989,
          0.00064457,  0.00061211]),
  'sampling_rate': 16000},
 'transcript': 'she was selected best young player awards in 2000 season',
 'age_group': '',
 'gender': 'Female',
 'accent': 'pidgin',
 'domain': 'general',
 'country': 'NG',
 'duration': 6.475986480712891}

#Whisper Model Loading

In [53]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = WhisperProcessor.from_pretrained(Whisper_DIR)
model = WhisperForConditionalGeneration.from_pretrained(Whisper_DIR, device_map="auto")
model.to(device)
model.eval()

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [54]:
def transcribe_whisper(audio_array, sr=16000):
    inputs = processor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        pred_ids = model.generate(**inputs)

    return processor.batch_decode(
        pred_ids, skip_special_tokens=True
    )[0].lower()

#Wav2Vec2 Model Loading

In [115]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [116]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor.from_pretrained(WAV_DIR)

from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(WAV_DIR)
model.to(device)
model.eval()

Wav2Vec2BertForCTC(
  (wav2vec2_bert): Wav2Vec2BertModel(
    (feature_projection): Wav2Vec2BertFeatureProjection(
      (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=160, out_features=1024, bias=True)
      (dropout): Dropout(p=0.001, inplace=False)
    )
    (encoder): Wav2Vec2BertEncoder(
      (dropout): Dropout(p=0.01, inplace=False)
      (layers): ModuleList(
        (0-23): 24 x Wav2Vec2BertEncoderLayer(
          (ffn1_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (ffn1): Wav2Vec2BertFeedForward(
            (intermediate_dropout): Dropout(p=0.0, inplace=False)
            (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
            (intermediate_act_fn): SiLU()
            (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
            (output_dropout): Dropout(p=0.01, inplace=False)
          )
          (self_attn_layer_norm): LayerNorm

In [117]:
def transcribe_wav2vec(audio_array, sr=16000):
    inputs = processor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    )

    input_features = inputs["input_features"].to(device)

    with torch.no_grad():
        logits = model(input_features=input_features).logits

    pred_ids = torch.argmax(logits, dim=-1)

    return processor.batch_decode(
        pred_ids, skip_special_tokens=True
    )[0].lower()

In [118]:
def extract_medical_tokens(text, term_set):
    tokens = text.lower().split()
    return [t for t in tokens if t in term_set]

In [119]:
import jiwer

def medical_wer(ref, hyp, term_set):
    ref_tokens = extract_medical_tokens(ref, term_set)
    hyp_tokens = extract_medical_tokens(hyp, term_set)

    if not ref_tokens:
        return None

    return jiwer.wer(
        " ".join(ref_tokens),
        " ".join(hyp_tokens)
    )

In [120]:
clinical_test_ds = [s for s in test_ds if s.get("domain") == "clinical"]
general_test_ds  = [s for s in test_ds if s.get("domain") == "general"]

print("Clinical:", len(clinical_test_ds))
print("General :", len(general_test_ds))

Clinical: 27
General : 18


In [121]:
from collections import defaultdict

stats = {
    "clinical": defaultdict(int),
    "general":  defaultdict(int)
}

mwer_values = {
    "clinical": [],
    "general": []
}

def evaluate_domain(ds, domain):
    for sample in ds:
        audio = sample["audio"]["array"]
        ref   = sample["transcript"].lower()
        hyp   = normalize_text(transcribe_wav2vec(audio))
        # hyp   = normalize_text(transcribe_whisper(audio))
        ref_terms = set(extract_medical_tokens(ref, medical_terms))
        hyp_terms = set(extract_medical_tokens(hyp, medical_terms))

        stats[domain]["tp"] += len(ref_terms & hyp_terms)
        stats[domain]["fp"] += len(hyp_terms - ref_terms)
        stats[domain]["fn"] += len(ref_terms - hyp_terms)

        w = medical_wer(ref, hyp, medical_terms)
        if w is not None:
            mwer_values[domain].append(w)

In [122]:
evaluate_domain(clinical_test_ds, "clinical")
evaluate_domain(general_test_ds, "general")

In [123]:
import numpy as np

def print_results(domain):
    tp = stats[domain]["tp"]
    fp = stats[domain]["fp"]
    fn = stats[domain]["fn"]

    precision = tp / (tp + fp) if tp + fp else 0
    recall    = tp / (tp + fn) if tp + fn else 0
    f1        = 2*precision*recall/(precision+recall) if precision+recall else 0
    mwer      = np.mean(mwer_values[domain])

    print(f"\n=== {domain.upper()} DOMAIN ===")
    print(f"Medical Precision : {precision:.3f}")
    print(f"Medical Recall    : {recall:.3f}")
    print(f"Medical F1        : {f1:.3f}")
    print(f"Medical WER       : {mwer:.3f}")

print_results("clinical")
print_results("general")


=== CLINICAL DOMAIN ===
Medical Precision : 0.942
Medical Recall    : 0.343
Medical F1        : 0.503
Medical WER       : 0.682

=== GENERAL DOMAIN ===
Medical Precision : 0.889
Medical Recall    : 0.800
Medical F1        : 0.842
Medical WER       : 0.250


In [124]:
def find_medical_term_errors(ref, hyp, medical_terms):
    ref_tokens = set(extract_medical_tokens(ref, medical_terms))
    hyp_tokens = set(extract_medical_tokens(hyp, medical_terms))

    missed_terms = ref_tokens - hyp_tokens   # false negatives
    extra_terms  = hyp_tokens - ref_tokens   # false positives

    return missed_terms, extra_terms

In [125]:
def utterance_wer(ref, hyp):
    if not isinstance(ref, str) or not isinstance(hyp, str):
        return np.nan
    if ref.strip() == "":
        return np.nan
    return jiwer.wer(ref, hyp)

In [126]:
error_samples = []

for i, sample in enumerate(clinical_test_ds):
    audio = sample["audio"]["array"]
    ref   = sample["transcript"].lower()
    hyp   = normalize_text(transcribe_wav2vec(audio))
    # hyp   = normalize_text(transcribe_whisper(audio))

    missed, extra = find_medical_term_errors(ref, hyp, medical_terms)

    wer_val = utterance_wer(ref, hyp)
    mwer_val = medical_wer(ref, hyp, medical_terms)

    error_samples.append({
        "sample_id": i,
        "reference": ref,
        "prediction": hyp,
        "WER": wer_val,
        "mWER": mwer_val,
        "missed_medical_terms": sorted(list(missed)),
        "extra_medical_terms": sorted(list(extra))
    })

In [127]:
def get_summary_metrics(domain):
    tp = stats[domain]["tp"]
    fp = stats[domain]["fp"]
    fn = stats[domain]["fn"]

    precision = tp / (tp + fp) if tp + fp else 0.0
    recall    = tp / (tp + fn) if tp + fn else 0.0
    f1        = 2 * precision * recall / (precision + recall) if precision + recall else 0.0
    mwer      = float(np.mean(mwer_values[domain])) if mwer_values[domain] else None

    return {
        "medical_precision": round(precision, 4),
        "medical_recall": round(recall, 4),
        "medical_f1": round(f1, 4),
        "medical_wer": round(mwer, 4) if mwer is not None else None,
        "tp": int(tp),
        "fp": int(fp),
        "fn": int(fn),
        "num_samples": int(len(clinical_test_ds) if domain == "clinical" else len(general_test_ds))
    }

In [128]:
results = {}

In [129]:
results["summary_metrics"] = {
    "clinical": get_summary_metrics("clinical"),
    "general":  get_summary_metrics("general"),
    "error_samples": error_samples,
}

In [130]:
results

{'summary_metrics': {'clinical': {'medical_precision': 0.9423,
   'medical_recall': 0.3427,
   'medical_f1': 0.5026,
   'medical_wer': 0.6822,
   'tp': 49,
   'fp': 3,
   'fn': 94,
   'num_samples': 27},
  'general': {'medical_precision': 0.8889,
   'medical_recall': 0.8,
   'medical_f1': 0.8421,
   'medical_wer': 0.25,
   'tp': 8,
   'fp': 1,
   'fn': 2,
   'num_samples': 18},
  'error_samples': [{'sample_id': 0,
    'reference': 'managementi n primary hyperparathyroidism surgical removal of the parathyroids is curative',
    'prediction': 'management and primary hyperpirodiroisin surgical rmoval of the pratoroids ist',
    'WER': 0.6363636363636364,
    'mWER': 0.8333333333333334,
    'missed_medical_terms': ['curative',
     'hyperparathyroidism',
     'managementi',
     'parathyroids',
     'removal'],
    'extra_medical_terms': []},
   {'sample_id': 1,
    'reference': 'minor anaphylactic reactions such as urticaria can be treated with oral or intramuscular chlorpheniramine',
   

In [131]:
len(results["summary_metrics"]["error_samples"])

27

In [132]:
OUTPUT_JSON = "/content/drive/MyDrive/project/Bert_Fair/Final_ASR_Medical_Error_Analysis_Fair.json"

with open(OUTPUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved results to {OUTPUT_JSON}")

Saved results to /content/drive/MyDrive/project/Bert_Fair/Final_ASR_Medical_Error_Analysis_Fair.json
